d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

-sandbox
# Hypothesis Testing (25 mins)

**Objective**: *Perform a two-sample, two-sided t-test to compare sample means.*

In this lab, you will complete a series of exercises to perform a t-test.

We want to determine whether the population mean of daily steps taken for the athlete health tracker users is equal to the population mean of daily steps taken for the cardio enthusiast health tracker users.

The null and alternative hypotheses for this test are:

* H<sub>0</sub> = the mean of daily steps taken for athlete users is equal to the mean of daily steps taken for cardio enthusiast users.
* H<sub>1</sub> = the mean of daily steps taken for athlete users is not equal to the mean of daily steps taken for cardio enthusiast users.

In [0]:
%run "./Includes/Classroom-Setup"

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Out[8]: DataFrame[]

res4: Boolean = false

res5: Boolean = false

## Exercise 1

Compute the sample mean of daily steps taken for sedentary users and cardio enthusiast users.

In [0]:
%sql

SELECT * FROM dsfda.ht_daily_metrics LIMIT 5

device_id,dte,resting_heartrate,active_heartrate,bmi,vo2,workout_minutes,lifestyle,steps
01c99836-e48b-11ea-8204-0242ac110002,2019-01-01,84.21990807741808,122.36614513322988,24.001442078098176,23.520174244131812,55.290771176379415,Athlete,10345
01c99836-e48b-11ea-8204-0242ac110002,2019-01-02,75.56009664616255,132.08213570805194,23.981473224271443,23.241395323502907,27.99672176043603,Athlete,9140
01c99836-e48b-11ea-8204-0242ac110002,2019-01-03,67.95214658754116,133.897419834453,24.01203323166733,23.844758959223647,6.403982740621206,Athlete,8895
01c99836-e48b-11ea-8204-0242ac110002,2019-01-04,72.07402249362427,130.27996931348147,24.184537499965806,23.65815389509837,21.06190942499232,Athlete,10683
01c99836-e48b-11ea-8204-0242ac110002,2019-01-05,84.21089219223238,130.45737338095134,24.504363282462865,23.930176012952693,11.171669668342071,Athlete,8752


In [0]:
%sql

SELECT DISTINCT lifestyle FROM dsfda.ht_daily_metrics 

lifestyle
Cardio Enthusiast
Athlete
Weight Trainer
Sedentary


In [0]:
%sql
-- TODO

SELECT lifestyle, avg(steps) AS mean
FROM dsfda.ht_daily_metrics
WHERE lifestyle = "Athlete" OR lifestyle = "Cardio Enthusiast"
GROUP BY lifestyle

lifestyle,mean
Cardio Enthusiast,13235.376990421259
Athlete,11001.597413366928


## Exercise 2
Compute the sample variance of daily steps taken for sedentary users and cardio enthusiast users.

In [0]:
%sql
-- TODO

SELECT lifestyle, var_samp(steps) AS variance
FROM dsfda.ht_daily_metrics
WHERE lifestyle = "Athlete" OR lifestyle = "Cardio Enthusiast"
GROUP BY lifestyle

lifestyle,variance
Cardio Enthusiast,1.3171492338618575E7
Athlete,9623550.234400436


## Exercise 3
Compute the sample size for sedentary users and cardio enthusiast users.

In [0]:
%sql
-- TODO

SELECT lifestyle, count(*) AS sample_size
FROM dsfda.ht_daily_metrics
WHERE lifestyle = "Sedentary" OR lifestyle = "Cardio Enthusiast"
GROUP BY lifestyle

lifestyle,sample_size
Cardio Enthusiast,388360
Sedentary,113880


In [0]:
%sql

SELECT lifestyle, count(*) AS sample_size
FROM dsfda.ht_daily_metrics
WHERE lifestyle = "Athlete"
GROUP BY lifestyle

lifestyle,sample_size
Athlete,313535


## Exercise 4
Compute the T-statistic using the sample statistics.

In [0]:
# TODO
from math import sqrt

athlete_mean = 11001.597413366928
athlete_variance = 9623550.2344004
athlete_size = 313535

cardio_mean = 13235.376990421259
cardio_variance = 13171492.338618632
cardio_size = 388360

test_statistic = (athlete_mean - cardio_mean) / sqrt((athlete_variance / athlete_size) + (cardio_variance / cardio_size))
print(f"T-statistic = {test_statistic}")

T-statistic = -277.90254777599523

## Exercise 5
Compute the degrees of freedom using the sample statistics.

In [0]:
# TODO
df_numerator = ((athlete_variance / athlete_size) + (cardio_variance / cardio_size))**2
df_denominator = (athlete_variance / athlete_size)**2 / (athlete_size - 1) + ((cardio_variance / cardio_size)**2 / (cardio_size - 1))
df = df_numerator / df_denominator
print(f"Degrees-of-freedom = {df}")

Degrees-of-freedom = 699615.065636117

## Exercise 6

Compute the p-value for this T-test by passing in the `test_statistic` and the `df` to `t.cdf()`.

In [0]:
# TODO
from scipy.stats import t
p_value = t.cdf(test_statistic, df)
print(f"p-value = {p_value}")

p-value = 0.0

-sandbox
## Exercise 7

Determine whether we should reject the null hypothesis. 

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Use a significance level of 0.05.

In [0]:
# TODO
print(f"Reject the null hypothesis as the p-value {p_value} is less than 0.05")

Reject the null hypothesis as the p-value 0.0 is less than 0.05

## Exercise 8

Phew! That was a lot of work to answer a simple question.

Luckily, Python's `scipy` module makes this process a bit easier than it already has.

Check out the demonstration below showing how to perform this same test in a single step only using Python.

In [0]:
from scipy.stats import ttest_ind

athlete_daily_steps = spark.sql("SELECT steps FROM dsfda.ht_daily_metrics WHERE lifestyle = 'Athlete'").toPandas()["steps"]
cardio_daily_steps = spark.sql("SELECT steps FROM dsfda.ht_daily_metrics WHERE lifestyle = 'Cardio Enthusiast'").toPandas()["steps"]

ttest_ind(athlete_daily_steps, cardio_daily_steps, equal_var = False)

Out[17]: Ttest_indResult(statistic=-277.9025477759952, pvalue=0.0)

Notice that the *same test statistic* and *same p-value* were calculated with far less code!

While it's good to understand how these `scipy` tools work, it's good practice and efficient to use them as much as possible.

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>